# Create Monday.com Report
-------

# Import Libraries

In [ ]:
import requests
import json
import csv
from monday import MondayClient
import datetime
from datetime import timedelta
import time

In [ ]:
import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
api_key = config['DEFAULT']['API-Key']
board_ids = config['DEFAULT']['Board-ID-List']
time_difference = config['DEFAULT']['Time-Difference']

# Supporting Functions

In [ ]:
def write_csv(output_file,row, mode):
    with open(output_file, mode, newline='') as csvfile:
        csvwriter = csv.writer(csvfile, quoting=csv.QUOTE_MINIMAL)
        csvwriter.writerow(row)

# Connect To Monday.com

In [ ]:
apiUrl = "https://api.monday.com/v2"
headers = {"Authorization" : api_key}
monday = MondayClient(api_key)

# Get Board ID List

In [ ]:
board_id_list = []
if board_ids:
    for b in board_ids.split(","):
        board_id_list.append(b)
else:
    query = '{ boards {id}}'
    data = {'query' : query}
    r = requests.post(url=apiUrl, json=data, headers=headers)
    for bd in r.json()["data"]["boards"]:
        board_id_list.append(bd["id"])

# Get Board.Group As Prefix For Job ID

In [ ]:
group_item_dict = {}
board_project_code_dict = {}

for board_id in board_id_list:
    board_query = '{{boards (ids: {}){{description name groups{{id}}}}}}'.format(board_id)
    board_data = {'query' : board_query}
    board_result = requests.post(url=apiUrl, json=board_data, headers=headers)
    
    project_code = board_result.json()["data"]["boards"][0]["description"].split(": ")[1]
    board_project_code_dict[board_id] = project_code
    
    board_name = board_result.json()["data"]["boards"][0]["name"]
    for g in board_result.json()["data"]["boards"][0]["groups"]:
        group_query = '{{boards (ids: {}){{groups(ids: {}) {{title items {{id}}}}}}}}'.format(board_id,g["id"])
        group_data = {'query' : group_query}
        group_result = requests.post(url=apiUrl, json=group_data, headers=headers)
        group_title = group_result.json()["data"]["boards"][0]["groups"][0]["title"]
        for i in group_result.json()["data"]["boards"][0]["groups"][0]["items"]:
            group_item_dict[i["id"]] = [board_name,group_title]

# Create End Of Year Report

In [ ]:
column_name = ["Staff Name", "Board Name", "Project Code", "Group Name", "Timesheet Date", "Week Number", "Time tracking (hrs)"]
write_csv('monday_report.csv',column_name,"w")

In [ ]:
datetimeFormat = '%Y-%m-%dT%H:%M:%SZ'

for b in board_id_list:
    group_name_person_time_dict = {}
    
    query = '{{items_by_column_values (board_id :{}, column_id: "status", column_value:"{}"){{id name column_values{{text title value}}}}}}'.format(b, "Timesheet Created")
    data = {'query' : query}
    r = requests.post(url=apiUrl, json=data, headers=headers)
    
    for v in r.json()["data"]["items_by_column_values"]:
        if v["id"] in group_item_dict:
            board_name = group_item_dict[v["id"]][0]
            group_name = group_item_dict[v["id"]][1]

        for cv in v["column_values"]:
            if cv["title"] == "Time Tracking":
                time_tracking = cv["text"]
                time_used = 0
                if time_tracking != "":
                    for av in json.loads(cv["value"])["additional_value"]:
                        # convert date string to datetime
                        start_at = datetime.datetime.strptime(av["started_at"], datetimeFormat)
                        end_at = datetime.datetime.strptime(av["ended_at"], datetimeFormat)

                        # change to correct time zone
                        start_at += datetime.timedelta(hours=int(time_difference))
                        end_at += datetime.timedelta(hours=int(time_difference))

                        work_day = str(end_at)[:10]

                        # calculate time range
                        diff = end_at - start_at
                        time_used += round(diff.seconds/3600, 2)

            if cv["title"] == "Timesheet date":
                timesheet_date = cv["text"]
                
            if cv["title"] == "Person":
                person = cv["text"]
        
        if timesheet_date:
            for p in person.split(", "):
                if group_name in group_name_person_time_dict:
                    if p in group_name_person_time_dict[group_name]:
                        if timesheet_date in group_name_person_time_dict[group_name][p]:
                            group_name_person_time_dict[group_name][p][timesheet_date] += time_used
                        else:
                            group_name_person_time_dict[group_name][p][timesheet_date] = {}
                            group_name_person_time_dict[group_name][p][timesheet_date] = time_used
                    else:
                        group_name_person_time_dict[group_name][p] = {}
                        group_name_person_time_dict[group_name][p][timesheet_date] = {}
                        group_name_person_time_dict[group_name][p][timesheet_date] = time_used
                else:
                    group_name_person_time_dict[group_name] = {}
                    group_name_person_time_dict[group_name][p] = {}
                    group_name_person_time_dict[group_name][p][timesheet_date] = {}
                    group_name_person_time_dict[group_name][p][timesheet_date] = time_used
    
    for group_name in group_name_person_time_dict:
        for person in group_name_person_time_dict[group_name]:
            for timesheet_date in group_name_person_time_dict[group_name][person]:
                week_number = datetime.date(int(timesheet_date[:4]), int(timesheet_date[5:7]), int(timesheet_date[-2:])).isocalendar()[1]
                row = [person,board_name,board_project_code_dict[b], group_name, timesheet_date, week_number, group_name_person_time_dict[group_name][person][timesheet_date]]
                write_csv("monday_report.csv",row,"a+")